In [1]:
import os
os.environ['PYSPARK_PYTHON'] = '/opt/conda/bin/python'
os.environ['PYSPARK_DRIVER_PYTHON'] = '/opt/conda/bin/python'

In [2]:
import datalabframework as dlf
import pyspark.sql.functions as F
import pyspark.sql.types as T
import yaml
import pandas as pd
from datetime import datetime, timedelta
from util_report.util_reports import *
from util_report.helpers import *
from util_report.util_email import *

In [3]:
start = datetime.now()

In [4]:
pd.set_option('display.max_columns', 30)

### Setup date

In [5]:
report_date_str = "2019-05-21"
format_str= "%Y-%m-%d"
report_date = datetime.strptime(report_date_str, format_str)
cube_date = report_date - timedelta(days=1)
first_of_last_month = (report_date.replace(day=1) - timedelta(days=1)).replace(day=1).replace(hour=0)
lastweek = (report_date - timedelta(days=7))
this_month = (report_date - timedelta(days=1)).month
this_year = (report_date - timedelta(days=1)).year

### Setup engine

In [6]:
dlf.project.load('default')
engine = dlf.project.engine()
spark = engine.context() 

### Standard channel list

In [7]:
MAPPING_CHANNELS = {'Showroom': 'Store', 'Apps': 'Agent-App'}

### Load cubes data to make data block

#### daily cube

In [8]:
# channel and customer province
pd_cube_revenue_by_channel = engine.load(path='daily/revenue_by_channel/_cube_date={}'.format(report_date.date()), provider='cube_hdfs')\
.withColumn('employee_channel', F.when(F.col('employee_channel').isin(["", "X"]), F.lit("Other")).otherwise(F.col('employee_channel'))).toPandas()

pd_cube_revenue_by_channel_lastweekend = engine.load(path='daily/revenue_by_channel/_cube_date={}'.format(lastweek.date()), provider='cube_hdfs')\
.withColumn('employee_channel', F.when(F.col('employee_channel').isin(["", "X"]), F.lit("Other")).otherwise(F.col('employee_channel'))).toPandas()

pd_daily_inventory_price_diff = engine.load(path='daily/daily_inventory_price_flutuation', provider='cube_hdfs')\
.filter((F.col('cube_date').isin([report_date.date(), lastweek.date()])))\
.toPandas()

pd_daily_revenue_by_customer_province = engine.load(path='daily/revenue_by_customer_province/_cube_date={}'.format(report_date.date()), provider='cube_hdfs')\
.withColumn('customer_province', title_string(F.col('customer_province'))).toPandas()

pd_daily_revenue_by_customer_province_lastweekend = engine.load(path='daily/revenue_by_customer_province/_cube_date={}'.format(lastweek.date()), provider='cube_hdfs')\
.withColumn('customer_province', title_string(F.col('customer_province'))).toPandas()

# profit rate
pd_cube_profit_rate_by_channel = engine.load(path='daily/profit_rate_by_channel/_cube_date={}'.format(report_date.date()), provider='cube_hdfs')\
.withColumn('employee_channel', F.when(F.col('employee_channel').isin(["", "X"]), F.lit("Other")).otherwise(F.col('employee_channel'))).toPandas()

pd_cube_profit_rate_by_channel_lastweekend = engine.load(path='daily/profit_rate_by_channel/_cube_date={}'.format(lastweek.date()), provider='cube_hdfs')\
.withColumn('employee_channel', F.when(F.col('employee_channel').isin(["", "X"]), F.lit("Other")).otherwise(F.col('employee_channel'))).toPandas()

pd_profit_rate_online = engine.load(path='daily/profit_rate_channel_online', provider='cube_hdfs')\
.filter(F.col('cube_date').isin([report_date.date(), lastweek.date()]))\
.toPandas()

pd_cube_profit_rate = engine.load(path='daily/profit_rate', provider='cube_hdfs')\
.filter(F.col('cube_date').isin([report_date.date(), lastweek.date()]))\
.toPandas()

# region area
pd_revenue_by_branch_region_code = engine.load(path='daily/revenue_by_branch_region_code/_cube_date={}'.format(report_date.date()), provider='cube_hdfs')\
.toPandas()

pd_revenue_by_branch_region_code_lastweekend = engine.load(path='daily/revenue_by_branch_region_code/_cube_date={}'.format(lastweek.date()), provider='cube_hdfs')\
.toPandas()

pd_inventory_stock_by_branch_region = engine.load(path='daily/inventory_stock_by_branch_region/_cube_date={}'.format(report_date.date()), provider='cube_hdfs')\
.toPandas()

pd_inventory_stock_by_branch_region_lastweekend = engine.load(path='daily/inventory_stock_by_branch_region/_cube_date={}'.format(lastweek.date()), provider='cube_hdfs')\
.toPandas()

# pd turnover all
pd_pd_turn_over_company = engine.load(path='daily/daily_inventory_turnover_7', provider='cube_hdfs')\
.filter(F.col('cube_date').isin([report_date.date(), lastweek.date()]))\
.toPandas()

In [9]:
# load cube for product
def load_product_cube(paths, provider='cube_hdfs'):
    list_path_by_date = ['_cube_date={}'.format(report_date.date()), '_cube_date={}'.format(lastweek.date())]
    
    list_by_cat_group = [engine.load(path='daily/{}/{}'.format(paths[0],x), provider=provider)\
        .withColumn('cat_group_id', F.when(F.col('cat_group_id').startswith('NGH04'), F.lit('NGH04')).otherwise(F.col('cat_group_id')))\
        .toPandas() for x in list_path_by_date]
    
    list_by_cat_root = [engine.load(path='daily/{}/{}'.format(paths[1],x), provider=provider)\
        .toPandas() for x in list_path_by_date]
    
    cube_cat_group = pd.concat(list_by_cat_group, sort=False)
    cube_cat_root = pd.concat(list_by_cat_root, sort=False)
    
    return cube_cat_group, cube_cat_root

pd_cube_revenue_by_cat_group, pd_cube_revenue_by_cat_root = load_product_cube(['revenue_by_cat_group', 'revenue_by_cat_root'])
pd_stock_by_cat_group, pd_stock_by_cat_root = load_product_cube(['inventory_stock_by_cat_group', 'inventory_stock_by_cat_root'])
pd_profit_rate_by_cat_group, pd_profit_rate_by_cat_root = load_product_cube(['profit_rate_by_cat_group', 'profit_rate_by_cat_root'])
pd_turn_over_by_cat_group, pd_turn_over_by_cat_root = load_product_cube(['daily_inventory_turnover_by_cat_group_7', 'daily_inventory_turnover_by_cat_root_7'])

#### monthly cube

In [10]:
pd_monthly_cube_revenue_by_channel = engine.load(path='monthly/monthly_revenue_by_channel/_cube_date={}-{}'.format(this_year, this_month), provider='cube_hdfs')\
.withColumn('employee_channel', F.when(F.col('employee_channel').isin(["", "X"]), F.lit("Other")).otherwise(F.col('employee_channel'))).toPandas()

pd_monthly_cube_revenue_by_customer_province = engine.load(path='monthly/monthly_revenue_by_customer_province/_cube_date={}-{}'.format(this_year, this_month), provider='cube_hdfs')\
.withColumn('customer_province', title_string(F.col('customer_province'))).toPandas()

pd_monthly_cube_profit_rate_by_channel = engine.load(path='monthly/monthly_profit_rate_by_channel/_cube_date={}-{}'.format(this_year, this_month), provider='cube_hdfs')\
.withColumn('employee_channel', F.when(F.col('employee_channel').isin(["", "X"]), F.lit("Other")).otherwise(F.col('employee_channel'))).toPandas()

pd_monthly_cube_profit_rate_channel_online = engine.load(path='monthly/monthly_profit_rate_channel_online/_cube_date={}-{}'.format(this_year, this_month), provider='cube_hdfs')\
.toPandas()

pd_monthly_cube_profit_rate = engine.load(path='monthly/monthly_profit_rate/_cube_date={}-{}'.format(this_year, this_month), provider='cube_hdfs')\
.toPandas()

#### reference data

In [11]:
# tinh
dmbp_tinh = engine.load(path='dmbp_tinh', provider='reference_data').select(
    F.col('province_name').alias('customer_province'), 'province_co_report', 'region_id'
).toPandas()

# danh muc nhom
dmnhom_rp = engine.load(path='dmnhom_rp', provider='reference_data').toPandas()

# kpi
kpi = engine.load(path='kpi', provider='reference_data')\
.filter('year == "{}"'.format(report_date.year))\
.select(F.col('channel').alias('employee_channel'), str_to_number(F.col('{}'.format(report_date.month))).cast('long').alias('ke_hoach_thang'))\
.toPandas()

In [12]:
kpi

,employee_channel,ke_hoach_thang
0,Showroom,187892890000
1,Chat,47000000000
2,Phone,22000000000
3,Apps,41366000000
4,Outsales,9500000000
5,Other,3000000000


In [13]:
kpi.employee_channel = [MAPPING_CHANNELS[channel] if channel in MAPPING_CHANNELS.keys() else channel for channel in kpi['employee_channel']]

In [14]:
kpi

,employee_channel,ke_hoach_thang
0,Store,187892890000
1,Chat,47000000000
2,Phone,22000000000
3,Agent-App,41366000000
4,Outsales,9500000000
5,Other,3000000000


### Get config

In [15]:
config = yaml.load(open('config_report/config_new.yml'))
config_list = config['eod'] 

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  """Entry point for launching an IPython kernel.


### Make revenue by channel

In [16]:
def combine_pd_this_day_lastweekend_and_month(group_cols, metric_col, pd_this_day, pd_lastweekend, pd_monthly, empty_col_val = ''):
    tmp_1 = pd_this_day.groupby(group_cols).sum()
    tmp_2 = pd_lastweekend.groupby(group_cols).sum()
    tmp_3 = pd_monthly.groupby(group_cols).sum()
    
    result = pd.concat([
        tmp_1[metric_col].rename('revenue'),
        tmp_2[metric_col].rename('last_week_revenue'),
        tmp_3[metric_col].rename('luy_ke_thang')
    ], axis=1, sort=True)
    
    # insert empty column at first
    result.insert(0, 'ngay', empty_col_val)
    
    return result

In [17]:
def factory_channel(list_cubes, group_col, metric_col, row_total_label, add_kpi=True):
    by_channel = combine_pd_this_day_lastweekend_and_month(
        [group_col], metric_col, *list_cubes
    )
    by_channel[group_col] = list(by_channel.index)
    if add_kpi:
        by_channel = pd.merge(by_channel, kpi, on=group_col, how='left').fillna(0)
        
    by_channel = by_channel.fillna(0).set_index(by_channel[group_col]).drop(columns=[group_col])
    
    if row_total_label:
        by_channel = add_sum_row(by_channel, row_label=row_total_label)    
        online_row = get_sum_row(by_channel.loc[by_channel.index != 'Store'], row_label='Online')
        by_channel = by_channel.append(online_row, sort=False)
    
    return by_channel

In [18]:
pd_all_revenue_by_channel = factory_channel(
    [pd_cube_revenue_by_channel, pd_cube_revenue_by_channel_lastweekend, pd_monthly_cube_revenue_by_channel],
    'employee_channel', 'sum_total_price', 'Doanh thu thuần (tỷ VND)'
)

In [19]:
pd_all_revenue_by_channel

,ngay,revenue,last_week_revenue,luy_ke_thang,ke_hoach_thang
Doanh thu thuần (tỷ VND),,7392764252.0000,6910331462.0000,182910510456.0000,3.01259e+11
Agent-App,,771660988.0000,585410910.0000,14818550468.0000,4.1366e+10
Chat,,948994969.0000,1000272008.0000,20146640324.0000,4.7e+10
Other,,19785001.0000,32481818.0000,258255186.0000,3e+09
Partner,,0,0,4940000.0000,0
Phone,,748080382.0000,697194095.0000,22547021823.0000,2.2e+10
Store,,4904242912.0000,4594972631.0000,125135102655.0000,1.87893e+11
Online,,9881285592.0000,9225690293.0000,240685918257.0000,4.14625e+11


In [20]:
# prepare to report
pd_all_revenue_by_channel.loc['Doanh thu thuần (tỷ VND)']['employee_channel'] = 'Doanh thu thuần (tỷ VND)'
pd_all_revenue_by_channel.loc['Online']['employee_channel'] = 'Online'
# pd_all_revenue_by_channel['% Kế hoạch'] = 100*pd_all_revenue_by_channel['luy_ke_thang'].astype('float')/pd_all_revenue_by_channel['ke_hoach_thang']
pd_all_revenue_by_channel['% Kế hoạch'] = list(map(lambda x: 100*x, custom_divide(pd_all_revenue_by_channel, 'luy_ke_thang', 'ke_hoach_thang')))
sorted_channels = get_list_sort_channel(pd_all_revenue_by_channel.index)
pd_all_revenue_by_channel = make_custom_sort_by_list_and_index(pd_all_revenue_by_channel, sorted_channels)
divide_cols(pd_all_revenue_by_channel, ['revenue', 'last_week_revenue', 'luy_ke_thang', 'ke_hoach_thang'], divide_unit=1000000000)
round_cols(pd_all_revenue_by_channel, ['revenue', 'last_week_revenue', 'luy_ke_thang', 'ke_hoach_thang', '% Kế hoạch'], 1)

In [21]:
pd_all_revenue_by_channel

,ngay,revenue,last_week_revenue,luy_ke_thang,ke_hoach_thang,% Kế hoạch
Doanh thu thuần (tỷ VND),,7.4,6.9,182.9,301.3,60.7
Store,,4.9,4.6,125.1,187.9,66.6
Online,,9.9,9.2,240.7,414.6,58.0
Agent-App,,0.8,0.6,14.8,41.4,35.8
Chat,,0.9,1.0,20.1,47.0,42.9
Other,,0.0,0.0,0.3,3.0,8.6
Partner,,0.0,0.0,0.0,0.0,0.0
Phone,,0.7,0.7,22.5,22.0,102.5


In [22]:
pd_profit_by_channel = factory_channel(
    [pd_cube_profit_rate_by_channel, pd_cube_profit_rate_by_channel_lastweekend, pd_monthly_cube_profit_rate_by_channel],
    'employee_channel', 'profit_rate', False, False
)

In [23]:
company_profit_rate = pd.DataFrame('', columns=pd_profit_by_channel.columns, index=["Tỷ suất lợi nhuận gộp (%)"])
company_profit_rate.loc["Tỷ suất lợi nhuận gộp (%)"]['revenue'] = pd_cube_profit_rate.loc[0]['profit_rate']
company_profit_rate.loc["Tỷ suất lợi nhuận gộp (%)"]['last_week_revenue'] = pd_cube_profit_rate.loc[1]['profit_rate']
company_profit_rate.loc["Tỷ suất lợi nhuận gộp (%)"]['luy_ke_thang'] = pd_monthly_cube_profit_rate.loc[0]['profit_rate']

In [24]:
online_profit = pd.DataFrame('', columns=pd_profit_by_channel.columns, index=["Online"])
online_profit.loc["Online"]['revenue'] = pd_profit_rate_online.loc[0]['profit_rate']
online_profit.loc["Online"]['last_week_revenue'] = pd_profit_rate_online.loc[1]['profit_rate']
online_profit.loc["Online"]['luy_ke_thang'] = pd_monthly_cube_profit_rate_channel_online.loc[0]['profit_rate']

In [25]:
pd_all_profit_by_channel = pd.concat([
    company_profit_rate,
    pd_profit_by_channel,
    online_profit
])

In [26]:
pd_all_profit_by_channel

,ngay,revenue,last_week_revenue,luy_ke_thang
Tỷ suất lợi nhuận gộp (%),,0.087571,0.082439,0.112919
Agent-App,,0.088301,0.078753,0.073057
Chat,,0.104031,0.059392,0.079427
Other,,0.152222,0.103334,0.095685
Partner,,0,0,-0.014078
Phone,,0.076059,0.093250,0.295527
Store,,0.085766,0.086138,0.090170
Online,,0.091128,0.075099,0.162192


In [27]:
sorted_channels_profit = get_list_sort_channel(pd_all_profit_by_channel.index)

In [28]:
pd_all_profit_by_channel = make_custom_sort_by_list_and_index(pd_all_profit_by_channel, sorted_channels_profit)

In [29]:
pd_all_profit_by_channel['ke_hoach_thang'] = ['']*len(pd_all_profit_by_channel.index)
pd_all_profit_by_channel['% Kế hoạch'] = ['']*len(pd_all_profit_by_channel.index)

In [30]:
divide_cols(pd_all_profit_by_channel, ['revenue', 'last_week_revenue', 'luy_ke_thang'], divide_unit=0.01)
round_cols(pd_all_profit_by_channel, ['revenue', 'last_week_revenue', 'luy_ke_thang'], 1)

In [31]:
pd_all_profit_by_channel.head()

,ngay,revenue,last_week_revenue,luy_ke_thang,ke_hoach_thang,% Kế hoạch
Tỷ suất lợi nhuận gộp (%),,8.8,8.2,11.3,,
Store,,8.6,8.6,9.0,,
Online,,9.1,7.5,16.2,,
Agent-App,,8.8,7.9,7.3,,
Chat,,10.4,5.9,7.9,,


In [32]:
all_df_revenue = pd.concat([
    pd_all_revenue_by_channel,
    pd_all_profit_by_channel
], sort=False)

In [33]:
all_df_revenue

,ngay,revenue,last_week_revenue,luy_ke_thang,ke_hoach_thang,% Kế hoạch
Doanh thu thuần (tỷ VND),,7.4,6.9,182.9,301.3,60.7
Store,,4.9,4.6,125.1,187.9,66.6
Online,,9.9,9.2,240.7,414.6,58
Agent-App,,0.8,0.6,14.8,41.4,35.8
Chat,,0.9,1.0,20.1,47,42.9
Other,,0.0,0.0,0.3,3,8.6
Partner,,0.0,0.0,0.0,0,0
Phone,,0.7,0.7,22.5,22,102.5
Tỷ suất lợi nhuận gộp (%),,8.8,8.2,11.3,,
Store,,8.6,8.6,9.0,,


In [34]:
pd_daily_inventory_price_diff.head()

,price_diff,cube_date,created_at,_cube_date
0,-0.456633,2019-05-21,2019-05-27 16:26:54.674765,2019-05-21
1,0.169053,2019-05-14,2019-05-27 16:23:47.482284,2019-05-14


In [35]:
df_bien_dong_gia = pd.DataFrame('', columns=all_df_revenue.columns, index=["Biến động giá hàng tồn kho (%)"]) 

In [36]:
df_bien_dong_gia['revenue'] = pd_daily_inventory_price_diff.loc[0]['price_diff']
df_bien_dong_gia['last_week_revenue'] = pd_daily_inventory_price_diff.loc[1]['price_diff']

In [37]:
df_bien_dong_gia

,ngay,revenue,last_week_revenue,luy_ke_thang,ke_hoach_thang,% Kế hoạch
Biến động giá hàng tồn kho (%),,-0.456633,0.169053,,,


In [38]:
round_cols(df_bien_dong_gia, ['revenue', 'last_week_revenue'], 1)

In [39]:
tmp_all_df_revenue = pd.concat([
    all_df_revenue,
    df_bien_dong_gia
], sort=False)

In [40]:
tmp_all_df_revenue

,ngay,revenue,last_week_revenue,luy_ke_thang,ke_hoach_thang,% Kế hoạch
Doanh thu thuần (tỷ VND),,7.4,6.9,182.9,301.3,60.7
Store,,4.9,4.6,125.1,187.9,66.6
Online,,9.9,9.2,240.7,414.6,58
Agent-App,,0.8,0.6,14.8,41.4,35.8
Chat,,0.9,1,20.1,47,42.9
Other,,0,0,0.3,3,8.6
Partner,,0,0,0,0,0
Phone,,0.7,0.7,22.5,22,102.5
Tỷ suất lợi nhuận gộp (%),,8.8,8.2,11.3,,
Store,,8.6,8.6,9,,


In [41]:
init_province = pd.DataFrame(get_list_sort_province(), columns=['customer_province'])

In [42]:
init_province.head()

,customer_province
0,Hà Nội
1,Nghệ An
2,Vĩnh Phúc
3,Bắc Giang
4,Thái Nguyên


In [43]:
tmp_province_1 = pd.merge(pd_daily_revenue_by_customer_province, dmbp_tinh, on='customer_province', how='inner')
tmp_province_2 = pd.merge(pd_daily_revenue_by_customer_province_lastweekend, dmbp_tinh, on='customer_province', how='inner')
tmp_province_3 = pd.merge(pd_monthly_cube_revenue_by_customer_province, dmbp_tinh, on='customer_province', how='inner')

tmp_province_1.drop_duplicates(subset="customer_province", keep="first", inplace=True)
tmp_province_2.drop_duplicates(subset="customer_province", keep="first", inplace=True)
tmp_province_3.drop_duplicates(subset="customer_province", keep="first", inplace=True)

In [44]:
tmp_province_1 = tmp_province_1.groupby(['province_co_report']).sum()
tmp_province_2 = tmp_province_2.groupby(['province_co_report']).sum()
tmp_province_3 = tmp_province_3.groupby(['province_co_report']).sum()

In [45]:
tmp_province_1['province_co_report'] = tmp_province_1.index

In [46]:
tmp_province_1.loc['KB']['customer_province'] = 'Các tỉnh khác miền Bắc'
tmp_province_1.loc['KT']['customer_province'] = 'Các tỉnh khác miền Trung'
tmp_province_1.loc['KN']['customer_province'] = 'Các tỉnh khác miền Nam'
tmp_province_2.loc['KB']['customer_province'] = 'Các tỉnh khác miền Bắc'
tmp_province_2.loc['KT']['customer_province'] = 'Các tỉnh khác miền Trung'
tmp_province_2.loc['KN']['customer_province'] = 'Các tỉnh khác miền Nam'
tmp_province_3.loc['KB']['customer_province'] = 'Các tỉnh khác miền Bắc'
tmp_province_3.loc['KT']['customer_province'] = 'Các tỉnh khác miền Trung'
tmp_province_3.loc['KN']['customer_province'] = 'Các tỉnh khác miền Nam'

In [47]:
from functools import reduce
list_pd_provinces = [init_province, tmp_province_1, tmp_province_2, tmp_province_3]
tong_hop_province = reduce(lambda left, right: pd.merge(left, right, on='customer_province'), list_pd_provinces)

In [48]:
tong_hop_province.head()

,customer_province,sum_total_price_x,sum_quantity_x,region_id_x,province_co_report,sum_total_price_y,sum_quantity_y,region_id_y,sum_total_price,sum_quantity,cube_date,region_id
0,Hà Nội,730840727.0000,570.0000,MBA,HN,548860147.0000,370.0000,MBA,12413402969.0000,8945.0000,2019-5,MBA
1,Nghệ An,18367273.0000,25.0000,MBA,NA,1072727.0000,4.0000,MBA,976946209.0000,1016.0000,2019-5,MBA
2,Vĩnh Phúc,10739090.0000,15.0000,MBA,VP,20527273.0000,9.0000,MBA,223749196.0000,180.0000,2019-5,MBA
3,Bắc Giang,4400000.0000,15.0000,MBA,BG,9740909.0000,5.0000,MBA,363014664.0000,244.0000,2019-5,MBA
4,Thái Nguyên,46820547.0000,24.0000,MBA,TNG,70524910.0000,59.0000,MBA,577698407.0000,452.0000,2019-5,MBA


In [49]:
tong_hop_province = tong_hop_province[['customer_province', 'region_id', 'sum_total_price_x', 'sum_total_price_y', 'sum_total_price']]

In [50]:
tong_hop_province = tong_hop_province.set_index(tong_hop_province.customer_province).drop(columns=['customer_province'])

In [51]:
tong_hop_province['ke_hoach_thang'] = tong_hop_province['% Kế hoạch'] = ''

In [52]:
tong_hop_province.head()

,region_id,sum_total_price_x,sum_total_price_y,sum_total_price,ke_hoach_thang,% Kế hoạch
customer_province,,,,,,
Hà Nội,MBA,730840727.0000,548860147.0000,12413402969.0000,,
Nghệ An,MBA,18367273.0000,1072727.0000,976946209.0000,,
Vĩnh Phúc,MBA,10739090.0000,20527273.0000,223749196.0000,,
Bắc Giang,MBA,4400000.0000,9740909.0000,363014664.0000,,
Thái Nguyên,MBA,46820547.0000,70524910.0000,577698407.0000,,


In [53]:
tong_hop_province_to_report = add_sum_row(
    tong_hop_province, row_label='Doanh thu thuần theo địa chỉ của khách hàng (triệu VND)'
)

In [54]:
tong_hop_province_to_report = change_values(
    tong_hop_province_to_report,
    ['Doanh thu thuần theo địa chỉ của khách hàng (triệu VND)'],
    ['customer_province', 'region_id']
)

In [55]:
tong_hop_province_to_report.loc['Các tỉnh khác miền Bắc']['region_id'] = 'MBA'
tong_hop_province_to_report.loc['Các tỉnh khác miền Trung']['region_id'] = 'MTR'
tong_hop_province_to_report.loc['Các tỉnh khác miền Nam']['region_id'] = 'MNA'

In [56]:
divide_cols(tong_hop_province_to_report, ['sum_total_price_x', 'sum_total_price_y', 'sum_total_price'], divide_unit=1000000000)
round_cols(tong_hop_province_to_report, ['sum_total_price_x', 'sum_total_price_y', 'sum_total_price'], 1)

In [57]:
tong_hop_province_to_report.head()

,region_id,sum_total_price_x,sum_total_price_y,sum_total_price,ke_hoach_thang,% Kế hoạch
Doanh thu thuần theo địa chỉ của khách hàng (triệu VND),,4.8,4.2,106.1,,
Hà Nội,MBA,0.7,0.5,12.4,,
Nghệ An,MBA,0.0,0.0,1.0,,
Vĩnh Phúc,MBA,0.0,0.0,0.2,,
Bắc Giang,MBA,0.0,0.0,0.4,,


In [58]:
tong_hop_province_to_report.columns = tmp_all_df_revenue.columns

In [59]:
final_table_revenue = pd.concat([
    tmp_all_df_revenue,
    tong_hop_province_to_report
], sort=False)

In [60]:
final_table_revenue

,ngay,revenue,last_week_revenue,luy_ke_thang,ke_hoach_thang,% Kế hoạch
Doanh thu thuần (tỷ VND),,7.4,6.9,182.9,301.3,60.7
Store,,4.9,4.6,125.1,187.9,66.6
Online,,9.9,9.2,240.7,414.6,58
Agent-App,,0.8,0.6,14.8,41.4,35.8
Chat,,0.9,1,20.1,47,42.9
Other,,0,0,0.3,3,8.6
Partner,,0,0,0,0,0
Phone,,0.7,0.7,22.5,22,102.5
Tỷ suất lợi nhuận gộp (%),,8.8,8.2,11.3,,
Store,,8.6,8.6,9,,


In [61]:
highlight_index_revenue = {
    'Doanh thu thuần (tỷ VND)': {}, 
    'Tỷ suất lợi nhuận gộp (%)': {},
    'Biến động giá hàng tồn kho (%)': {},
    'Doanh thu thuần theo địa chỉ của khách hàng (triệu VND)': {}
}

In [62]:
header_revenue = {
    'parent': 'headers',
    'level': 'level1',
    'type': 'single'
}

In [63]:
all_df_revenue_jinja = get_data_before_render(config_list, final_table_revenue, report_date, header_revenue, highlight_index=highlight_index_revenue)

In [64]:
all_df_revenue_html = render_content('table.html', {'data': all_df_revenue_jinja})

### Region code report

In [65]:
revenue_region_code = pd.concat([
    pd_revenue_by_branch_region_code['sum_total_price'].rename('this_day_sum_total_price'),
    pd_revenue_by_branch_region_code_lastweekend['sum_total_price'].rename('lastweekend_sum_total_price'),
], axis=1, sort=False)

In [66]:
revenue_region_code = revenue_region_code.set_index(pd_revenue_by_branch_region_code['branch_region_code'])

In [67]:
revenue_region_code

,this_day_sum_total_price,lastweekend_sum_total_price
branch_region_code,,
MTR,476095659.0000,517932980.0000
MBA,1542936790.0000,882684639.0000
MNA,5373731803.0000,5509713843.0000


In [68]:
inventory_region_code = pd.concat([
    pd_inventory_stock_by_branch_region['sum_end_day_value'].rename('this_day_sum_end_day_value'),
    pd_inventory_stock_by_branch_region_lastweekend['sum_end_day_value'].rename('lastweekend_sum_end_day_value')
], axis=1, sort=False)

In [69]:
inventory_region_code

,this_day_sum_end_day_value,lastweekend_sum_end_day_value
0,28140000.000000,6395001076.311789
1,141518273430.843886,31145118654.280758
2,3760336007.162526,596248182.955000
3,31205724083.730643,157939092367.164528


In [70]:
inventory_region_code = inventory_region_code.set_index(pd_inventory_stock_by_branch_region['branch_region_code'])

In [71]:
inventory_region_code

,this_day_sum_end_day_value,lastweekend_sum_end_day_value
branch_region_code,,
NaN,28140000.000000,6395001076.311789
MNA,141518273430.843886,31145118654.280758
MTR,3760336007.162526,596248182.955000
MBA,31205724083.730643,157939092367.164528


In [72]:
tmp_all_region_code = pd.concat([
    revenue_region_code,
    inventory_region_code
], axis=1, sort=False)

In [73]:
tmp_all_region_code

,this_day_sum_total_price,lastweekend_sum_total_price,this_day_sum_end_day_value,lastweekend_sum_end_day_value
MTR,476095659.0000,517932980.0000,3760336007.162526,596248182.955000
MBA,1542936790.0000,882684639.0000,31205724083.730643,157939092367.164528
MNA,5373731803.0000,5509713843.0000,141518273430.843886,31145118654.280758
NaN,NaN,NaN,28140000.000000,6395001076.311789


In [74]:
tmp_all_region_code.insert(0, 'area_description', [
    'Miền Trung (bao gồm cả kho tổng)', 'Miền Nam (bao gồm cả TT bảo hành và kho tổng)', 'Miền Bắc (bao gồm cả kho tổng)', 'Không xác định'
])

In [75]:
tmp_all_region_code

,area_description,this_day_sum_total_price,lastweekend_sum_total_price,this_day_sum_end_day_value,lastweekend_sum_end_day_value
MTR,Miền Trung (bao gồm cả kho tổng),476095659.0000,517932980.0000,3760336007.162526,596248182.955000
MBA,Miền Nam (bao gồm cả TT bảo hành và kho tổng),1542936790.0000,882684639.0000,31205724083.730643,157939092367.164528
MNA,Miền Bắc (bao gồm cả kho tổng),5373731803.0000,5509713843.0000,141518273430.843886,31145118654.280758
NaN,Không xác định,NaN,NaN,28140000.000000,6395001076.311789


In [76]:
all_region_code = add_sum_row(tmp_all_region_code, row_label='Toàn công ty')

In [77]:
all_region_code

,area_description,this_day_sum_total_price,lastweekend_sum_total_price,this_day_sum_end_day_value,lastweekend_sum_end_day_value
Toàn công ty,Miền Trung (bao gồm cả kho tổng)Miền Nam (bao ...,7392764252.0000,6910331462.0000,176512473521.737055,196075460280.712075
MTR,Miền Trung (bao gồm cả kho tổng),476095659.0000,517932980.0000,3760336007.162526,596248182.955000
MBA,Miền Nam (bao gồm cả TT bảo hành và kho tổng),1542936790.0000,882684639.0000,31205724083.730643,157939092367.164528
MNA,Miền Bắc (bao gồm cả kho tổng),5373731803.0000,5509713843.0000,141518273430.843886,31145118654.280758
NaN,Không xác định,NaN,NaN,28140000.000000,6395001076.311789


In [78]:
all_region_code.loc['Toàn công ty']['area_description'] = ''

In [79]:
all_region_code.insert(1, 'ngay', '')

In [80]:
all_region_code.fillna(0)

,area_description,ngay,this_day_sum_total_price,lastweekend_sum_total_price,this_day_sum_end_day_value,lastweekend_sum_end_day_value
Toàn công ty,,,7392764252.0000,6910331462.0000,176512473521.737055,196075460280.712075
MTR,Miền Trung (bao gồm cả kho tổng),,476095659.0000,517932980.0000,3760336007.162526,596248182.955000
MBA,Miền Nam (bao gồm cả TT bảo hành và kho tổng),,1542936790.0000,882684639.0000,31205724083.730643,157939092367.164528
MNA,Miền Bắc (bao gồm cả kho tổng),,5373731803.0000,5509713843.0000,141518273430.843886,31145118654.280758
NaN,Không xác định,,0,0,28140000.000000,6395001076.311789


In [81]:
divide_cols(all_region_code, [
    'this_day_sum_total_price', 'lastweekend_sum_total_price', 'this_day_sum_end_day_value', 'lastweekend_sum_end_day_value'
], divide_unit=1000000000)
round_cols(all_region_code, [
    'this_day_sum_total_price', 'lastweekend_sum_total_price', 'this_day_sum_end_day_value', 'lastweekend_sum_end_day_value'
], 1)

In [82]:
all_region_code

,area_description,ngay,this_day_sum_total_price,lastweekend_sum_total_price,this_day_sum_end_day_value,lastweekend_sum_end_day_value
Toàn công ty,,,7.4,6.9,176.5,196.1
MTR,Miền Trung (bao gồm cả kho tổng),,0.5,0.5,3.8,0.6
MBA,Miền Nam (bao gồm cả TT bảo hành và kho tổng),,1.5,0.9,31.2,157.9
MNA,Miền Bắc (bao gồm cả kho tổng),,5.4,5.5,141.5,31.1
NaN,Không xác định,,NaN,NaN,0.0,6.4


In [83]:
highlight_index_area_code = {
    'Toàn công ty': {}
}
headers_region = {
    'parent': 'second_headers',
    'type': 'multiple'
}

In [84]:
all_region_code_jinja = get_data_before_render(config_list, all_region_code, report_date, headers_region, highlight_index=highlight_index_area_code)

In [85]:
### By Product

In [86]:
def get_rp_by_cat_group_and_cat_root(cat_group, cat_root, values):
    by_cat_group = cat_group.pivot_table(
        index='cat_group_id', 
        columns='cube_date', 
        values=values, 
        aggfunc='sum',
    )

    by_cat_root = cat_root.pivot_table(
            index='cat_root_id', 
            columns='cube_date', 
            values=values, 
            aggfunc='sum',
        )

    return pd.concat([
        by_cat_group,
        by_cat_root
    ])

In [87]:
# prepare data
report_revenue_by_product = get_rp_by_cat_group_and_cat_root(pd_cube_revenue_by_cat_group, pd_cube_revenue_by_cat_root, 'sum_total_price')
report_stock_by_product = get_rp_by_cat_group_and_cat_root(pd_stock_by_cat_group, pd_stock_by_cat_root, 'sum_end_day_value')
report_profit_by_product = get_rp_by_cat_group_and_cat_root(pd_profit_rate_by_cat_group, pd_profit_rate_by_cat_root, 'profit_rate')
vk_by_product = get_rp_by_cat_group_and_cat_root(pd_turn_over_by_cat_group, pd_turn_over_by_cat_root, 'vong_quay')

In [88]:
# all product report info
all_report_revenue_by_product = pd.concat([
    report_stock_by_product,
    vk_by_product,
    report_revenue_by_product,
    report_profit_by_product,
], axis=1, sort=False)

# add column nhom_cha to join
all_report_revenue_by_product['nhom_cha'] = [nhom for nhom in all_report_revenue_by_product.index]

In [89]:
# after that, rename columns
all_report_revenue_by_product.columns = [
    'inventory_this_week', 'inventory_last_week', 'turn_over_this_week', 'turn_over_last_week',
    'net_revenue_this_week', 'net_revenue_last_week', 'gross_margin_this_week', 'gross_margin_last_week', 'nhom_cha'
]

In [90]:
all_report_revenue_by_product.head()

,inventory_this_week,inventory_last_week,turn_over_this_week,turn_over_last_week,net_revenue_this_week,net_revenue_last_week,gross_margin_this_week,gross_margin_last_week,nhom_cha
NGH01,16364651131.891000,26637598029.550000,55.893705,52.775205,2657245390.0000,2516024344.0000,0.053761,0.052275,NGH01
NGH02,75449224748.076769,54778686680.778846,110.994783,145.178303,2228530405.0000,2535882530.0000,0.103132,0.113812,NGH02
NGH03,24359914242.060414,25062074830.893069,125.836922,121.042575,1090666044.0000,1352356656.0000,0.097889,0.103456,NGH03
NGH04,7527390584.702756,8738097348.774000,94.197543,94.297698,697640593.0000,633979696.0000,0.336946,0.387377,NGH04
NGH05,442895152.960000,554189434.528000,13.452760,34.483563,19979997.0000,23989279.0000,0.052582,0.172251,NGH05


In [91]:
tmp_all_report_revenue_by_product_rp = pd.merge(
    dmnhom_rp[['nhom_cha', 'nganh_report_co', 'nganh_en', 'don_vi']], all_report_revenue_by_product, on='nhom_cha', how='left'
).groupby('nganh_report_co').sum()

In [92]:
tmp_all_report_revenue_by_product_rp = add_sum_row(tmp_all_report_revenue_by_product_rp, row_label='Toàn công ty')

In [93]:
tmp_all_report_revenue_by_product_rp.head()

,nhom_cha,nganh_en,don_vi,inventory_this_week,inventory_last_week,turn_over_this_week,turn_over_last_week,net_revenue_this_week,net_revenue_last_week,gross_margin_this_week,gross_margin_last_week
Toàn công ty,01-N00101-N00201-N00402-N00102-N00202-N00302-N...,LaptopLaptop AccessoriesApple productPCMainboa...,MáyCáiCáiBộCáiCáiCáiCáiCáiCáiCáiCáiBộCáiCáiCái...,321971561048.712014,294205227997.114970,8410.444554,8211.659068,13794266558.0000,14745828407.0000,9.719557,35.621258
01-N001,01-N001,Laptop,Máy,14357346995.336000,24404820394.915000,2.774557,2.532394,2366784505.0000,2333754032.0000,0.048784,0.039200
01-N002,01-N002,Laptop Accessories,Cái,1266999784.191000,1298184711.089000,7.712194,7.040790,121549075.0000,139055772.0000,0.180972,0.226967
01-N004,01-N004,Apple product,Cái,0.000000,62400000.000000,0.000000,-0.098581,149181818.0000,16181818.0000,0.003656,0.112360
02-N001,02-N001,PC,Bộ,2685264058.630769,4444490131.985846,3.447546,2.928958,508124743.0000,478587049.0000,0.059204,0.075856


In [94]:
all_report_revenue_by_product_rp = make_custom_sort_by_cat(tmp_all_report_revenue_by_product_rp)

In [95]:
pd_pd_turn_over_company

,vong_quay,cube_date,created_at,_cube_date
0,12.422787,2019-05-21,2019-05-27 16:36:17.392560,2019-05-21
1,10.462343,2019-05-14,2019-05-27 16:32:38.866580,2019-05-14


In [96]:
all_report_revenue_by_product_rp.loc['Toàn công ty']['nganh_en'] = 'Product'
all_report_revenue_by_product_rp.loc['Toàn công ty']['don_vi'] = 'Unit'
all_report_revenue_by_product_rp.loc['Toàn công ty']['turn_over_this_week'] = pd_pd_turn_over_company.loc[0]['vong_quay']
all_report_revenue_by_product_rp.loc['Toàn công ty']['turn_over_last_week'] = pd_pd_turn_over_company.loc[1]['vong_quay']
all_report_revenue_by_product_rp.loc['Toàn công ty']['gross_margin_this_week'] = pd_cube_profit_rate.loc[0]['profit_rate']
all_report_revenue_by_product_rp.loc['Toàn công ty']['gross_margin_last_week'] = pd_cube_profit_rate.loc[1]['profit_rate']
for index in all_report_revenue_by_product_rp.index:
    if 'KHAC' in index:
        all_report_revenue_by_product_rp.loc[index]['nganh_en'] = 'Others'
        
all_report_revenue_by_product_rp = all_report_revenue_by_product_rp.drop(columns=['nhom_cha'])

In [97]:
all_report_revenue_by_product_rp['gross_margin_this_week'] *= 100
all_report_revenue_by_product_rp['gross_margin_last_week'] *= 100

divide_cols(all_report_revenue_by_product_rp, [
    'inventory_this_week', 'inventory_last_week'
], divide_unit=1000000000)
divide_cols(all_report_revenue_by_product_rp, [
    'net_revenue_this_week', 'net_revenue_last_week'
])
round_cols(all_report_revenue_by_product_rp, [
    'inventory_this_week', 'inventory_last_week', 'net_revenue_this_week', 'net_revenue_last_week', 
    'turn_over_this_week', 'turn_over_last_week', 'gross_margin_this_week', 'gross_margin_last_week'
], 1)

In [98]:
all_report_revenue_by_product_rp.head()

,nganh_en,don_vi,inventory_this_week,inventory_last_week,turn_over_this_week,turn_over_last_week,net_revenue_this_week,net_revenue_last_week,gross_margin_this_week,gross_margin_last_week
Toàn công ty,Product,Unit,322.0,294.2,12.4,10.5,13794.3,14745.8,8.8,8.2
NGH01,Laptop & Accessories,,16.4,26.6,55.9,52.8,2657.2,2516.0,5.4,5.2
01-N001,Laptop,Máy,14.4,24.4,2.8,2.5,2366.8,2333.8,4.9,3.9
01-N002,Laptop Accessories,Cái,1.3,1.3,7.7,7.0,121.5,139.1,18.1,22.7
01-N004,Apple product,Cái,0.0,0.1,0.0,-0.1,149.2,16.2,0.4,11.2


In [99]:
highlight_index_product = {
    'Toàn công ty': {},
    'NGH04': {},
    'NGH01': {},
    'NGH02/03': {},
    'NGH03': {},
    'NGH05': {},
    'NGH06': {},
    'NGH07': {},
    'NGH08': {},
    'NGH09': {},
    'NGH-KHAC': {}
}
headers_product = {
    'parent': 'third_headers',
    'type': 'multiple'
}

In [100]:
all_report_revenue_by_product_jinja = get_data_before_render(
    config_list, 
    all_report_revenue_by_product_rp, 
    report_date, 
    headers_product, 
    highlight_index=highlight_index_product
)

In [101]:
content = ""
content += render_content('header.html', {
    'style': get_styles('eod.yml'), 
    'data': {
        'title': 'BÁO CÁO KẾT QUẢ KINH DOANH',
        'extras_information': {
            'Loại': 'HÀNG NGÀY',
            'Level': 'CÔNG TY'
        }
    }
})
content += render_content('table.html', {'data': all_df_revenue_jinja})
content += "<br/><br/>"
content += render_content('table.html', {'data': all_region_code_jinja})
content += "<br/><br/>"
content += render_content('table.html', {'data': all_report_revenue_by_product_jinja})

In [102]:
html = ''.join(content)
from yattag import indent
html_pretty = indent(
    html,
    indentation = '    ',
    newline = '\r\n',
    indent_text = True
)
#Write to file 
f = open('eod_new.html', "wb")
f.write(str(html_pretty).encode('utf-8'))
f.close()
print("write done")

write done
